In [1]:
#include "TdrStyle.C"

In [2]:
// files information
TString pathData = "/home/choij/Desktop/CMS/plotMaker/FakeEstimator/2016/RunSysts__/DATA/";
TString pathMC = "/home/choij/Desktop/CMS/plotMaker/FakeEstimator/2016/RunSysts__/";
TString AnalyzerName = "FakeEstimator_";
vector<TString> SampleName 
    = {"DoubleEG", "MC", "DYJets", "tt", "SingleTop", "tW", "WJets_MG", "VV"};
TString suffix = ".root";
TString Year = "_2016";

map<TString, TFile*> files; files.clear();

for (int i = 0; i < SampleName.size(); i++) {
    TString sample = "";
    if (SampleName.at(i).Contains("DoubleEG")){
        sample = pathData + AnalyzerName + SampleName.at(i) + Year + suffix;
    }
    else {
        sample = pathMC + AnalyzerName + SampleName.at(i) + suffix;
    }
    TFile* temp = new TFile(sample);
    files.insert(make_pair(SampleName.at(i), temp));
}

// Working points and systematic sources
vector<TString> WPs = {"Loose", "Tight"};
vector<TString> Systs = {"Central", "Syst_FlavorDependence", "Syst_JetPtCutDown", "Syst_JetPtCutUp"};

// Histograms
vector<TString> Hists = {
    "M(ee)_Z_enriched", "Mt_W_enriched", "corrPt_QCD_enriched",
    "electron_eta_QCD_enriched", "electron_eta_W_enriched", "electron_pt_W_enriched",
    "leading_electron_eta_Z_enriched", "leading_electron_pt_Z_enriched", "leading_jet_eta_QCD_enriched",
    "leading_jet_eta_W_enriched", "leading_jet_eta_Z_enriched", "leading_jet_pt_QCD_enriched",
    "leading_jet_pt_W_enriched", "leading_jet_pt_Z_enriched", "subleading_electron_eta_Z_enriched",
    "subleading_electron_pt_Z_enriched", "subleading_jet_eta_QCD_enriched", "subleading_jet_pt_QCD_enriched",
    "nEvent_QCD_enriched_pass"
};

In [3]:
// function to get a directory
TDirectory* GetDirectory(TFile* file, TString wp, TString syst) {
    TDirectory* temp = (TDirectory*)file->GetDirectory(wp + "_" + syst);
    return temp;
}

In [4]:
auto GetHist(TDirectory* dir, TString histname) {
    auto* hist = dir->Get(histname);
    return hist;
}

In [5]:
// function to get a scale factor
Double_t GetScale(TString wp, TString syst) {
    TDirectory* dir_data = GetDirectory(files["DoubleEG"], wp, syst); 
    TDirectory* dir_mc = GetDirectory(files["MC"], wp, syst);
    
    TH1D* h_data = (TH1D*) dir_data->Get("Mt_W_enriched_" + wp + "_" + syst);
    TH1D* h_mc = (TH1D*) dir_mc->Get("Mt_W_enriched_" + wp + "_" + syst);
    
    Double_t scale = h_data->Integral() / h_mc->Integral();
    
    return scale;
}

In [6]:
THStack* GetStack(TString wp, TString syst, TString histname) {
    THStack* stack = new THStack(histname + "_" + wp + "_" + syst, "");
    
    for (int i = 2; i < SampleName.size(); i++) {
        TDirectory* dir 
            = (TDirectory*)GetDirectory(files[SampleName.at(i)], wp, syst);
        TH1D* hist = (TH1D*)GetHist(dir, histname + "_" + wp + "_" + syst);
        hist->Scale(GetScale(wp, syst));
         
        if (SampleName.at(i) == "tt") {
            hist->SetFillColorAlpha(kBlue, 1.);
            hist->SetLineColor(38);
        }
        else if (SampleName.at(i) == "DYJets") {
            hist->SetFillColorAlpha(kGreen, 0.7);
            hist->SetLineColor(kGreen);
        }
        else if (SampleName.at(i) == "SingleTop") {
            hist->SetFillColorAlpha(kBlue, 0.7);
            hist->SetLineColor(4);
        }
        else if (SampleName.at(i) == "tW") {
            hist->SetFillColorAlpha(kBlue, 0.4);
            hist->SetLineColor(9);
        }
        else if (SampleName.at(i) == "WJets_MG") {
            hist->SetFillColorAlpha(kAzure, 0.7);
            hist->SetLineColor(kAzure);
        }
        else if (SampleName.at(i) == "VV") {
            hist->SetFillColorAlpha(kRed, 0.7);
            hist->SetLineColor(kRed);
        }
        else cout << "This BkgProc(" << SampleName.at(i) << ") does not have matched color!" << endl;
        
        stack->Add(hist);
    }   
    return stack;
}

In [7]:
THStack* hstack = NULL;
hstack = GetStack("Tight", "Central", Hists.at(11));

In [ ]:
TCanvas* cvs = new TCanvas();
cvs->cd();
hstack->Draw("histfsame");
cvs->Draw();


 *** Break *** segmentation violation


In [ ]:
TCanvas* cvs = new TCanvas("cvs", "cvs", 800, 600);
TPad* pad_up = new TPad("pad_up", "pad_up", 0, 0.3, 1, 1);
TPad* pad_down = new TPad("pad_down", "pad_down", 0, 0, 1, 0.25);



In [ ]:
TDirectory* dir = GetDirectory(files["DoubleEG"], "Tight", "Central");
dir->ls();

In [ ]:
files["tt"]->ls();